<a href="https://colab.research.google.com/github/Jamal811/Generative-AI/blob/main/Multi_Article_Q%26A_Chatbot_with_File_Upload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1: Install Required Libraries

In [41]:
!pip install transformers torch pymupdf sentence-transformers faiss-cpu gradio

Step 2: Import Libraries

In [31]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer
import fitz  # PyMuPDF for PDF handling
import faiss
import numpy as np

In [32]:
# Load the embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Load a generative model for answering questions based on context
qa_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [33]:
def upload_and_extract_text():
    uploaded_files = files.upload()
    documents = {}

    for file_name in uploaded_files.keys():
        if file_name.endswith(".pdf"):
            with fitz.open(file_name) as pdf:
                text = ""
                for page in pdf:
                    text += page.get_text("text")
                documents[file_name] = text
        else:
            print(f"{file_name} is not a PDF.")

    print("PDF content extracted successfully.")
    return documents

# Split document text into chunks and embed them
def embed_documents(documents, chunk_size=300):
    passages, file_names, indexes = [], [], []

    for file_name, text in documents.items():
        # Break document into smaller chunks
        chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
        passages.extend(chunks)
        file_names.extend([file_name] * len(chunks))
        indexes.extend(range(len(chunks)))

    # Create embeddings
    embeddings = embedding_model.encode(passages, convert_to_tensor=True, show_progress_bar=True)

    return passages, embeddings, file_names, indexes

# Load and process PDFs
documents = upload_and_extract_text()
passages, embeddings, file_names, indexes = embed_documents(documents)

Saving Enhancing_News_Summarization_based_on_Advanced_Deep_Learning_Models_using_the_BBC_News_Dataset.pdf to Enhancing_News_Summarization_based_on_Advanced_Deep_Learning_Models_using_the_BBC_News_Dataset (5).pdf
PDF content extracted successfully.


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [34]:
# Convert embeddings to numpy and index with FAISS
embedding_matrix = embeddings.cpu().numpy()
index = faiss.IndexFlatL2(embedding_matrix.shape[1])
index.add(embedding_matrix)

In [36]:
def retrieve_relevant_passages(question, top_k=3):
    # Embed the question
    question_embedding = embedding_model.encode([question])

    # Retrieve relevant passages
    distances, retrieved_indices = index.search(np.array(question_embedding), top_k)
    retrieved_passages = [passages[i] for i in retrieved_indices[0]]
    return retrieved_passages

def answer_question(question, top_k=3):
    retrieved_passages = retrieve_relevant_passages(question, top_k)
    context = " ".join(retrieved_passages)  # Combine passages for context

    # Generate answer
    input_text = f"Answer the question based on this content: {context}. Question: {question}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    output_ids = qa_model.generate(input_ids, max_length=150)
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return answer

In [38]:
# Example question
question = "What are the objectives of this document?"
answer = answer_question(question)
print("Answer:", answer)

Answer: a summary can make news articles easier to understand for people who struggle with complex language or lengthy text, such as non- native speakers or those with cognitive problems


In [39]:
# Example question
question = "Which model has best performance?"
answer = answer_question(question)
print("Answer:", answer)

Answer: XLNet-Base-Cased


In [40]:
import gradio as gr
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
import fitz  # PyMuPDF for PDF handling
import faiss
import numpy as np

# Load models for embeddings and QA
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
qa_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

# Global variables to store documents and index
documents, passages, embeddings, file_names, indexes, index = {}, [], None, [], [], None

# Function to extract text from uploaded PDFs
def upload_and_extract_text(files):
    global documents
    documents = {}

    for file in files:
        with fitz.open(file.name) as pdf:
            text = ""
            for page in pdf:
                text += page.get_text("text")
            documents[file.name] = text
    return "PDF content extracted and indexed successfully."

# Function to embed documents and create FAISS index
def embed_and_index_documents(chunk_size=300):
    global passages, embeddings, file_names, indexes, index

    passages, file_names, indexes = [], [], []

    for file_name, text in documents.items():
        chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
        passages.extend(chunks)
        file_names.extend([file_name] * len(chunks))
        indexes.extend(range(len(chunks)))

    # Create embeddings
    embeddings = embedding_model.encode(passages, convert_to_tensor=False)
    embedding_matrix = np.array(embeddings)

    # Build FAISS index
    index = faiss.IndexFlatL2(embedding_matrix.shape[1])
    index.add(embedding_matrix)
    return "Documents embedded and indexed successfully."

# Function to retrieve relevant passages
def retrieve_relevant_passages(question, top_k=3):
    question_embedding = embedding_model.encode([question])
    distances, retrieved_indices = index.search(np.array(question_embedding), top_k)
    retrieved_passages = [passages[i] for i in retrieved_indices[0]]
    return retrieved_passages

# Function to answer questions using retrieved passages
def answer_question(question, top_k=3):
    retrieved_passages = retrieve_relevant_passages(question, top_k)
    context = " ".join(retrieved_passages)
    input_text = f"Answer the question based on this content: {context}. Question: {question}"

    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    output_ids = qa_model.generate(input_ids, max_length=150)
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return answer

# Gradio interface functions
def handle_file_upload(files):
    message = upload_and_extract_text(files)
    indexing_message = embed_and_index_documents()
    return f"{message}\n{indexing_message}"

def chat_with_pdfs(question):
    answer = answer_question(question)
    return answer

# Define Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("# PDF Chatbot using RAG (Retrieval-Augmented Generation)")

    with gr.Tab("Upload PDF(s)"):
        file_upload = gr.File(label="Upload PDF files", file_types=[".pdf"], file_count="multiple")
        upload_button = gr.Button("Process PDFs")
        upload_output = gr.Textbox(label="Status")
        upload_button.click(fn=handle_file_upload, inputs=file_upload, outputs=upload_output)

    with gr.Tab("Chat with PDFs"):
        question_input = gr.Textbox(label="Ask a question about the uploaded PDFs")
        answer_output = gr.Textbox(label="Answer")
        question_input.submit(fn=chat_with_pdfs, inputs=question_input, outputs=answer_output)

# Launch the app
demo.launch()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://eb4161ab2b0ec82b96.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


share=True